In [21]:
import numpy as np
import pandas as pd

# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [141]:
df = pd.read_csv("../data/adult.csv", index_col=[0]).replace("?", pd.NA).dropna()
df.head()

,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K,39
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K,35
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K,27
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K,43
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K,25


In [138]:
print(df["salary"].isin(["<=50K", ">50K"]).all())
print(df["salary K$"].astype(str).str.isnumeric().all())
print((df.loc[df["salary"] == ">50K", "salary K$"] > 50).all())
print((df.loc[df["salary"] == "<=50K", "salary K$"] <= 50).all())

True
True
True
True


In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30162 entries, 0 to 32560
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             30162 non-null  int64 
 1   workclass       30162 non-null  object
 2   education       30162 non-null  object
 3   marital-status  30162 non-null  object
 4   occupation      30162 non-null  object
 5   relationship    30162 non-null  object
 6   race            30162 non-null  object
 7   sex             30162 non-null  object
 8   hours-per-week  30162 non-null  int64 
 9   native-country  30162 non-null  object
 10  salary          30162 non-null  object
 11  salary K$       30162 non-null  int64 
dtypes: int64(3), object(9)
memory usage: 3.0+ MB


# Task 1
Print the count of men and women in the dataset.

In [147]:
df["sex"].value_counts()

sex
Male      20380
Female     9782
Name: count, dtype: int64

# Task 2
Find the average age of men in dataset

In [157]:
df[df["sex"] == "Male"]["age"].mean()

39.18400392541707

# Task 3
Get the percentage of people from Poland (native-country)

In [249]:
(df["native-country"] == "Poland").mean()

0.0018566408063125787

# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [178]:
print(df[df["salary"] == ">50K"]["age"].mean())
print(df[df["salary"] == ">50K"]["age"].std())
print(df[df["salary"] == "<=50K"]["age"].mean())
print(df[df["salary"] == "<=50K"]["age"].std())

43.95911028236548
10.269632835673852
36.60806038668668
13.464631257161633


# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [350]:
(
    df[
        ~df["education"].isin(["Bachelors", "Prof-school", "Assoc-acdm", "Assoc-voc", "Masters", "Doctorate"])
        & (df["salary"] == ">50K")
    ]["education"]
).count()

3178

# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [351]:
df.groupby("education")["age"].describe()

,count,mean,std,min,25%,50%,75%,max
education,,,,,,,,
10th,820.0,37.897561,16.225795,17.0,23.0,36.0,52.0,90.0
11th,1048.0,32.363550,15.089307,17.0,18.0,28.5,43.0,90.0
12th,377.0,32.013263,14.373710,17.0,19.0,28.0,41.0,79.0
1st-4th,151.0,44.622517,14.929051,19.0,33.0,44.0,56.0,81.0
5th-6th,288.0,41.649306,14.754622,17.0,28.0,41.0,53.0,82.0
7th-8th,557.0,47.631957,15.737479,17.0,34.0,49.0,60.0,90.0
9th,455.0,40.303297,15.335754,17.0,28.0,38.0,53.0,90.0
Assoc-acdm,1008.0,37.286706,10.509755,19.0,29.0,36.0,44.0,90.0
Assoc-voc,1307.0,38.246366,11.181253,19.0,30.0,37.0,45.0,84.0


# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [352]:
married = df[df["marital-status"].str.startswith("Married") & (df["sex"] == "Male")].value_counts("salary")
non_married = df[~df["marital-status"].str.startswith("Married") & (df["sex"] == "Male")].value_counts("salary")

In [359]:
married

salary
<=50K    7052
>50K     5723
Name: count, dtype: int64

In [360]:
non_married

salary
<=50K    6932
>50K      673
Name: count, dtype: int64

# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [361]:
max_hours = df["hours-per-week"].max()
print("Max hours per week:", max_hours)
print(df[df["hours-per-week"] == max_hours]["hours-per-week"].count(), "people")

Max hours per week: 99
78 people


# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight their connection.

In [362]:
# The older people have the higher salary and work more hours, salary depends on working hours per week
df.select_dtypes(pd.Int64Dtype).corr()

,age,hours-per-week,salary K$
age,1.000000,0.101599,0.208203
hours-per-week,0.101599,1.000000,0.196378
salary K$,0.208203,0.196378,1.000000
